# KAKENの研究種目マスタをローカルのMariaDBに保存するプログラム

### 事前準備
- KAKENマスタデータは、git のリポジトリで管理されており、最新のデータを利用可能。
- https://bitbucket.org/niijp/grants_masterxml_kaken/ からリポジトリを pull して、ローカルの ./grants_masterxml_kaken フォルダに同期しておく。
- ローカルで MariaDB を動かしておく。
- MariaDB のユーザ名、パスワード、データベース名は、config.ipynbを使って ./config.ini として保存しておく。

### ここから本編
bitbucketから読み込んだマスタのXMLファイルをelementTreeに変換

In [34]:
# encoding: utf-8
from lxml import etree
import pandas as pd

In [35]:
tree = etree.parse('grants_masterxml_kaken/category_master_kakenhi.xml')

categorylist = []
for category_table in tree.iterfind("category_table"):
    for category in category_table.iterfind("category"):
        name = category.find("name[@lang='ja']").text
        niicode = category.find("code[@type='nii']").text

        row = [
            niicode,
            name,
        ]
        categorylist.append(row)            

df = pd.DataFrame(categorylist)
df.columns = ['category_niicode', 'category_name']
df

,category_niicode,category_name
0,1,機関研究
1,2,各個研究
2,3,特定研究
3,4,総合研究
4,5,試験研究
5,6,海外学術調査
6,7,研究成果刊行費
7,8,奨励研究
8,1,機関研究
9,2,各個研究


重複のデータがあるかどうか

In [36]:
df.duplicated().any()

True

重複データがあった。dropしておく。

In [37]:
df = df.drop_duplicates()
df.duplicated().any()

False

In [38]:
df.category_niicode.value_counts()

41     1
57     1
72     1
51     1
5      1
26     1
9      1
20     1
31     1
66     1
14     1
73     1
17     1
76     1
47     1
44     1
70     1
34     1
39     1
18     1
4      1
32     1
33     1
54     1
74     1
16     1
25     1
65     1
45     1
62     1
      ..
64     1
23     1
30     1
75     1
36     1
46     1
29     1
53     1
249    1
56     1
10     1
58     1
69     1
68     1
42     1
77     1
13     1
40     1
19     1
48     1
60     1
15     1
2      1
247    1
12     1
55     1
71     1
37     1
43     1
250    1
Name: category_niicode, Length: 80, dtype: int64

niicodeがユニークなので、インデックスに設定する

In [39]:
df = df.set_index('category_niicode')
df

ローカルのmariaDBに関する設定ファイルを読み込み（config.iniはgitに上げていないが、事前準備で作られているはず）

In [41]:
import configparser

config = configparser.ConfigParser()
config.read('config.ini')
username = config['mariadb']['username']
password = config['mariadb']['password']
database = config['mariadb']['database']
url = 'mysql+pymysql://' + username + ':' + password + '@localhost:3306/' + database + '?charset=utf8'

データベースにテーブル構造とデータの中身を書き込む

In [43]:
from sqlalchemy import create_engine
from sqlalchemy.types import String, Integer

engine = create_engine(url, echo=True)

df.to_sql('kaken_master_category', engine, if_exists='replace',
          dtype={
              'category_niicode': Integer,
              'category_name': String(256),
                })

2018-05-09 12:00:52,742 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2018-05-09 12:00:52,743 INFO sqlalchemy.engine.base.Engine {}
2018-05-09 12:00:52,745 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2018-05-09 12:00:52,746 INFO sqlalchemy.engine.base.Engine {}
2018-05-09 12:00:52,748 INFO sqlalchemy.engine.base.Engine show collation where `Charset` = 'utf8' and `Collation` = 'utf8_bin'
2018-05-09 12:00:52,749 INFO sqlalchemy.engine.base.Engine {}
2018-05-09 12:00:52,751 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2018-05-09 12:00:52,752 INFO sqlalchemy.engine.base.Engine {}
2018-05-09 12:00:52,754 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2018-05-09 12:00:52,755 INFO sqlalchemy.engine.base.Engine {}
2018-05-09 12:00:52,756 INFO sqlalchemy.engine.base.Engine SELECT CAST('test collated returns' AS CHAR CHARACTER SET utf8) COLLATE utf8_bin AS anon_1
2018-05-09 12

### おしまい
データがコミットされていれば終了。HeidiSQLなどで、上記ドロップ済みデータフレームの件数が登録されているか確認する。